# Workflow

In [ ]:
import mne
import numpy as np 
import matplotlib.pyplot as plt
import torch
import os
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

## Downloading the Data

To download the data, we'll use the Data folder which contains ways to download data. Here we'll start off by using the EEG_BCI motor imagery dataset.

If we want to download the entire dataset, we can do the following:

In [ ]:
from Data import download_EEGBCI

if input("Are you sure you want to download the entire dataset? (Y/N)") in "Yy":
    total_subjects = list(range(1, 110))
    runs = list(range(1, 15))
    download_EEGBCI(total_subjects, runs, './EEGData', False)
    print("Downloaded everything!")

else:
    print("Download cancelled.")


# Only downloading a subset of the dataset
subjects = [x for x in range(1,10)]
runs = [1, 2, 3]
download_EEGBCI(subjects, runs, './EEGData', False)

## Loading the Data

This step is critical for actually getting the data into the ML algorithms. To do this, we'll have to rely on MNE's import functions to load the data into a format that we can use.

In [ ]:
main_folder = './EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0'

subdirectories = [f.path for f in os.scandir(main_folder) if f.is_dir()]

opened_files = []
closed_files = []

for subdirectory in subdirectories:
    files = os.listdir(subdirectory)
    
    if len(files) > 0:
        for file in files:
            
            if file[-6:] == '01.edf':
                # This is data for eyes opened
                eyes_opened = os.path.join(subdirectory, file)
                print(eyes_opened)
                opened_files.append(eyes_opened)

            if file[-6:] == '02.edf':
                # This is data for eyes closed
                eyes_closed = os.path.join(subdirectory, file)
                print(eyes_closed)
                closed_files.append(eyes_closed)
    else:
        print(f"No files found in {subdirectory}")